In [8]:
import os
import multiprocessing
import logging
from typing import Optional
import sys
import itertools

import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import math

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [9]:
# Load netcdf with wind speeds
file = '/home/mark/projects/open-gira/results/direct/Bahamas/max_wind_field_IBTrACS_BHS_0.02.nc'
ds = xr.open_dataset(file)
# Adjust the lat / lon coordinates to ensure they are counting cell corners (currently doing cell centres)
ds = ds.assign_coords({
    'longitude': ds.coords['longitude'] - 0.05,
    'latitude': ds.coords['latitude'] + 0.05
})
ds

<xarray.Dataset>
Dimensions:         (event_id: 58, latitude: 549, longitude: 518)
Coordinates:
  * event_id        (event_id) object '2001255N26276' ... '2022311N21293'
  * latitude        (latitude) float64 18.5 18.52 18.54 ... 29.42 29.44 29.46
  * longitude       (longitude) float64 -81.54 -81.52 -81.5 ... -71.22 -71.2
Data variables:
    spatial_ref     int64 ...
    max_wind_speed  (event_id, latitude, longitude) float64 ...

In [10]:
# Add event_ids to list (so that we can iterate over the storms)
storms = ds.coords['event_id'].values.tolist()

In [12]:
# Load the static datasets
gdp_data = rasterio.open('/home/mark/projects/open-gira/results/direct/Bahamas/GDP2005_0.02_BHS.tif')
natID_data = rasterio.open('/home/mark/projects/open-gira/results/direct/Bahamas/gpw4_natID_0.02_BHS.tif')
raster_meta = gdp_data.meta
raster_transform = gdp_data.transform
gdp_raster = gdp_data.read(1)
natID_raster = natID_data.read(1)
tc_country_damage_lookup = pd.read_csv('/home/mark/projects/open-gira/results/direct/Bahamas/TC_country_damage_look_up.csv')
damage_functions = pd.read_csv('/home/mark/projects/open-gira/results/direct/Bahamas/Eberenz_damage_functions.csv')

In [13]:
# Define functions

# To speed up operations we want to look up in dictionaries rather than dataframes
# Convert DataFrame to dictionary for faster lookup
damage_dict = damage_functions.set_index('Label')['v_half_tdr'].to_dict() # Use TDR optimization as per Eberenz et. al (2021)
tc_country_damage_dict = tc_country_damage_lookup.set_index('ISO')['damage'].to_dict()

def damage_function(wind_speed, value, v_thresh, v_half):
    '''
    Given a wind_speed, GDP value, and damage function factors, returns a GDP damage value
    '''
    
    numerator = max([(wind_speed - v_thresh), 0])
    denominator = v_half - v_thresh
    v_n = numerator/denominator
    fractional_damage = math.pow(v_n, 3)/(1+math.pow(v_n, 3))
    damage = fractional_damage * value

    return damage

def apply_damage_function(gdp_value, wind_speed, national_ID):
    '''
    Lookup damage function factors based on national_ID - then apply damage function to all cells.
    '''
    basin = tc_country_damage_dict.get(str(national_ID))
    if basin:
        v_half = damage_dict.get(basin)
        if v_half is not None:
            damage = damage_function(wind_speed, gdp_value, 25.7, v_half)
    else:
        damage = None

    return damage

# Vectorize damage function
vectorized_damage_function = np.vectorize(apply_damage_function)

In [14]:
### Loop through storms applying damage function and saving results
# Info for saving results
country_ids = tc_country_damage_lookup['ISO'].unique()
country_names = tc_country_damage_lookup.set_index('ISO')['Name'].to_dict()
columns = ['Event ID', 'Total Damage'] + [country_names[id] for id in country_ids]
results_list = [] 
for i in storms:
    tc_event = ds['max_wind_speed'].sel(event_id=str(i)).to_numpy()
    tc_damage = vectorized_damage_function(gdp_raster, tc_event, natID_raster)

    # Create a dictionary to hold summary data for this storm
    summary_data = {'Event ID': i, 'Total Damage': np.nansum(tc_damage)}
    
    # Loop through each country
    for country_id in country_ids:
        # mask damage array to include only cells for this country and sum damage
        if country_id == '999 - not UN defined':
            country_mask = np.where(natID_raster == 999, 1, 0)
        else:
            country_mask = np.where(natID_raster == int(country_id), 1, 0)
        country_damage = np.nansum(tc_damage * country_mask)
        country_name = country_names[country_id]
        summary_data[country_name] = country_damage

    # Append to results list
    results_list.append(summary_data)

    # Debug (print damage rasters) ###############

    output_raster = '/home/mark/projects/open-gira/results/direct/Bahamas/inspect/storm_damages_%s.tif' % i

    # Update metadata for the output raster
    raster_meta.update({
        "dtype": 'float32',  # Update to float32 (or match tc_damage.dtype)
        "count": 1  # Single-band raster
    })

    # Write the array to a GeoTIFF file
    with rasterio.open(output_raster, "w", **raster_meta) as dest:
        dest.write(tc_damage.astype('float32'), 1)

# Create a results dataframe
results_df = pd.DataFrame(results_list, columns=columns)

In [7]:
results_df.to_csv('/home/mark/projects/open-gira/results/direct/Bahamas/BHS_IBTrACS_0.02_results.csv')